In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [25]:
api_key = 'RGAPI-cdb07b88-033f-456a-ad59-d62a01b3c858'

In [26]:
sohwan = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" +'hide on bush' +'?api_key=' + api_key
r = requests.get(sohwan)
r.json()['id'] #소환사의 고유 id

'a2kiPGzdwTTGVP72yqU3m8tk1ofosuNIaHqyOHgHsv-jCA'

In [27]:
tier_url = "https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/" + r.json()['id'] +'?api_key=' + api_key
r2  = requests.get(tier_url)
r2.json()

[]

챌린저 리그 데이터 수집

In [19]:
# 챌린저 리그 게임데이터 수집
challenger = 'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
r = requests.get(challenger)#챌린저 호출
challenger_league_df = pd.DataFrame(r.json())

In [21]:

challenger_league_df.reset_index(inplace=True)#수집한 챌린저데이터 index정리
challenger_league_entries_df = pd.DataFrame(dict(challenger_league_df['entries'])).T #dict구조로 되어 있는 entries컬럼 풀어주기
challenger_league_df = pd.concat([challenger_league_df, challenger_league_entries_df], axis=1) #열끼리 결합

challenger_league_df = challenger_league_df.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)
challenger_league_df

,index,tier,entries


In [9]:
# 4. 소환사 고유 닉네임(summonerName)를 이용하여 유저별 고유 puuId 수집
for i in range(len(challenger_league_df)):
    try:
        sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + challenger_league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
        r = requests.get(sohwan)
        
        while r.status_code == 429:
            time.sleep(5)
            sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + challenger_league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
            r = requests.get(sohwan)
            
         
        challenger_league_df["puuid"] = r.json()['puuid']
    
    except:
        pass

In [11]:
challenger_league_df.to_csv('챌린저데이터.csv',index=False,encoding = 'cp949')#중간저장

In [ ]:
# 5. 매치데이터 수집
#13시즌의 데이터만을 이용할 것이며 league_df3 => 기존에 수집한 account_id가 있는 league_df
match_info_df = pd.DataFrame()
season = str(13)
for i in range(len(challenger_league_df)):
    try:
        match0 = 'https://kr.api.riotgames.com/lol/match/v5/matchlists/by-account/' + challenger_league_df['account_id'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
        r = requests.get(match0)
        
        while r.status_code == 429:
            time.sleep(5)
            match0 = 'https://kr.api.riotgames.com/lol/match/v5/matchlists/by-account/' + challenger_league_df['account_id'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
            r = requests.get(match0)
            
        match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])
    
    except:
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
#6. MatchId(gameId)를 활용한 최종 경기 데이터 수집

for i in range(len(match_info_df)):    
    
    api_url='https://kr.api.riotgames.com/lol/match/v4/matches/' + str(match_info_df['gameId'].iloc[i]) + '?api_key=' + api_key
    r = requests.get(api_url)

    if r.status_code == 200: # response가 정상이면 바로 맨 밑으로 이동하여 정상적으로 코드 실행
        pass

    elif r.status_code == 429:
        print('api cost full : infinite loop start')
        print('loop location : ',i)
        start_time = time.time()

        while True: # 429error가 끝날 때까지 무한 루프
            if r.status_code == 429:

                print('try 10 second wait time')
                time.sleep(10)

                r = requests.get(api_url)
                print(r.status_code)

            elif r.status_code == 200: #다시 response 200이면 loop escape
                print('total wait time : ', time.time() - start_time)
                print('recovery api cost')
                break

    elif r.status_code == 503: # 잠시 서비스를 이용하지 못하는 에러
        print('service available error')
        start_time = time.time()

        while True:
            if r.status_code == 503 or r.status_code == 429:

                print('try 10 second wait time')
                time.sleep(10)

                r = requests.get(api_url)
                print(r.status_code)

            elif r.status_code == 200: # 똑같이 response가 정상이면 loop escape
                print('total error wait time : ', time.time() - start_time)
                print('recovery api cost')
                break
    elif r.status_code == 403: # api갱신이 필요
        print('you need api renewal')
        print('break')
        break

    # 위의 예외처리 코드를 거쳐서 내려왔을 때 해당 코드가 실행될 수 있도록 작성
    mat = pd.DataFrame(list(r.json().values()), index=list(r.json().keys())).T
    match_fin = pd.concat([match_fin,mat])

In [ ]:
# 7. 최종 Team, GameDuration을 병합하여 데이터 셋 구축
#챔피언아이디를 제외하고 딕셔너리를 뽑는다

a_ls = list(data['teams'])
#team1
team1_df = pd.DataFrame()
for i in range(len(a_ls)):
    try:
        a_ls[i][0].pop('bans',None)
        team1 = pd.DataFrame(list(a_ls[i][0].values()),index = list(a_ls[i][0].keys())).T
        team1_df = team1_df.append(team1)
    except:
        pass
    
team1_df.index = range(len(team1_df))

#team2
team2_df = pd.DataFrame()
for i in range(len(a_ls)):
    try:
        a_ls[i][1].pop('bans',None)
        team2 = pd.DataFrame(list(a_ls[i][1].values()),index = list(a_ls[i][1].keys())).T
        team2_df = team2_df.append(team2)
    except:
        pass
    
team2_df.index = range(len(team2_df))

#컬럼으로 풀어준 team1과 team2와 duration의 데이터를 합쳐준다.
data_team = pd.concat([team1_df,team2_df,data[['gameDuration']]],axis=1)

그랜드마스터 리그 데이터 수집

In [ ]:
grandmaster = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
r = requests.get(grandmaster)#그마데이터 호출
grandmaster_league_df = pd.DataFrame(r.json())